# Spark SQL
Before we can write and run SQL queries, we need to tell Spark to treat the DataFrame as a SQL table. Spark internally maintains a virtual database within the SQLContext object. This object, which we enter as sqlCtx, has methods for registering temporary tables.

To register a DataFrame as a table, call the registerTempTable() method on that DataFrame object. This method requires one string parameter, name, that we use to set the table name for reference in our SQL queries.

1. Use the registerTempTable() method to register the DataFrame df as a table named census2010.
    - Then, run the SQLContext method tableNames to return the list of tables.
2. Assign the resulting list to tables, and use the print function to display it.

In [3]:
import findspark
findspark.init()
# Import PySpark and initialize SparkContext object.
import pyspark
sc = pyspark.SparkContext.getOrCreate()
from pyspark.sql import SQLContext
sqlCtx = SQLContext(sc)
df = sqlCtx.read.json("data/census_2010.json")

In [10]:
df.registerTempTable('census2010')
tables = sqlCtx.tableNames()
print(tables)

['census2010']


Now that we've registered the table within sqlCtx, we can start writing and running SQL queries. With Spark SQL, we represent our query as a string and pass it into the sql() method within the SQLContext object. The sql() method requires a single parameter, the query string. Spark will return the query results as a DataFrame object. This means you'll have to use show() to display the results, due to lazy loading.

While SQLite requires that queries end with a semi-colon, Spark SQL will actually throw an error if you include it. Other than this difference in syntax, Spark's flavor of SQL is identical to SQLite, and all the queries you've written for the SQL course will work here as well.
1. Write a SQL query that returns the age column from the table census2010, and use the show() method to display the first 20 results.

In [21]:
query = "select age from census2010"
sqlCtx.sql(query).show()

+---+
|age|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
| 15|
| 16|
| 17|
| 18|
| 19|
+---+
only showing top 20 rows



SQL shines at expressing complex logic in a more compact manner. Let's brush up on SQL by writing a query that expresses more specific criteria.


1. Write and run a SQL query that returns:
2. The males and females columns (in that order) where age > 5 and age < 15

In [20]:
query = 'select males, females from census2010 where age > 5 and age <15'
sqlCtx.sql(query).describe().show()

+-------+------------------+------------------+
|summary|             males|           females|
+-------+------------------+------------------+
|  count|                 9|                 9|
|   mean|         2124558.0|2031901.5555555555|
| stddev|33572.118476497715|26969.739459768203|
|    min|           2093905|           2007781|
|    max|           2195773|           2089651|
+-------+------------------+------------------+



One of the most powerful use cases in SQL is joining tables. Spark SQL takes this a step further by enabling you to run join queries across data from multiple file types. Spark will read any of the file types and formats it supports into DataFrame objects and we can register each of these as tables within the SQLContext object to use for querying.

As we mentioned briefly in the previous mission, most data science organizations use a variety of file formats and data storage mechanisms. Spark SQL was built with the industry use cases in mind and enables data professionals to use one common query language, SQL, to interact with lots of different data sources. We'll explore joins in Spark SQL further, but first let's introduce the other datasets we'll be using:
<pre>
census_1980.json - 1980 U.S. Census data
census_1990.json - 1990 U.S. Census data
census_2000.json - 2000 U.S. Census data
</pre>
1. Read these additional datasets into DataFrame objects and then use the registerTempTable() function to register these tables individually within SQLContext:
    - census_1980.json as census1980,
    - census_1990.json as census1990,
    - census_2000.json as census2000.
2. Then use the method tableNames() to list the tables within the SQLContext object, assign to tables, and finally print tables.

In [12]:
df1980 = sqlCtx.read.json("data/census_1980.json")
df1990 = sqlCtx.read.json("data/census_1990.json")
df2000 = sqlCtx.read.json("data/census_2000.json")
df1980.registerTempTable('census1980')
df1990.registerTempTable('census1990')
df2000.registerTempTable('census2000')
sqlCtx.tableNames()

['census1980', 'census1990', 'census2000', 'census2010']

1. Write a query that returns a DataFrame with the total columns for the tables census2010 and census2000 (in that order).
2. Then, run the query and use the show() method to display the first 20 results.

In [15]:
query = "select a.total, b.total from census2010 a join census2000 b on a.age = b.age"
sqlCtx.sql(query).take(2)

[Row(total=4079669, total=3733034), Row(total=4085341, total=3825896)]

The functions and operators from SQLite that we've used in the past are available for us to use in Spark SQL:

- COUNT()
- AVG()
- SUM()
- AND
- OR

1. Write a query that calculates the sums of the total column from each of the tables, in the following order:
<pre>
census2010,
census2000,
census1990.
</pre>
You'll need to perform two inner joins for this query (all datasets have the same values for age, which makes things convenient for joining).

In [19]:
query = "select sum(a.total) as total2010, sum(b.total) as total2000, sum(c.total) as total1990 from census2010 a  join \
census2000 b on a.age = b.age join census1990 c on b.age = c.age"
sqlCtx.sql(query).take(2)

[Row(total2010=312247116, total2000=284594395, total1990=254506647)]